In [1]:
# comment out this block 
# import os
# from dotenv import load_dotenv
# 
# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")

In [65]:
# model
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.agents import create_pandas_dataframe_agent

# we may have multiple datasets
paths = "D:/syjnl/Documents/GenAI/AI project/genAI/Soccer.xlsx"      # depending on upload process, don't need to be this form
df = pd.read_excel(paths)

# set temperature = 0 so that we get deterministic answer
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=512, openai_api_key = api_key)

# extract the python code for the user question
agent = create_pandas_dataframe_agent(
    llm,
    df,
    agent_type = "tool-calling",
    allow_dangerous_code = True,
    verbose = False)

def extract_python(text): 
    lines = text.splitlines()
    idx = lines.index("```python")
    return "\n".join(lines[idx:])

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding code and results, present the code and answer the user question in a clear, conversational style.
    
    Question: {question}
    Code: {code}
    Result: {result}
    
    Answer:""")

rephrase_answer = answer_prompt | llm | StrOutputParser()

print("Ask anything (type 'exit' to quit):", '\n')
while True:
    q = input(">>")
    if q.lower() in {"exit", "quit"}:
        break
    try:
        raw = agent.invoke(q + " Provide the executed code as well.")
        print("Raw GPT-4o output:", '\n', raw, '\n')
        
        code = extract_python(raw["output"])
        print("Executing Python: ", '\n', code, '\n')
        
        result = raw["output"].split('\n')[0]
        print("Raw Result: ", '\n', result, '\n')
        
        # 9. Rephrase into human-friendly language
        human_answer = rephrase_answer.invoke({
            "question": q,
            "code": code,
            "result": result
        })
        print("Rephrased Answer:", '\n', human_answer, "\n")

    except Exception as err:
        print("Error:", err, "\n")

Ask anything (type 'exit' to quit): 

Raw GPT-4o output: 
 {'input': 'how many games did Man City and Man United win in total? Provide the executed code as well.', 'output': "Man United won 9 games, and Man City won 13 games in total. \n\nHere is the executed code:\n\n```python\nman_united_wins = df[(df['home'] == 'Man United') & (df['result_class'] == 'W')].shape[0] + df[(df['away'] == 'Man United') & (df['result_class'] == 'L')].shape[0]\nman_city_wins = df[(df['home'] == 'Man City') & (df['result_class'] == 'W')].shape[0] + df[(df['away'] == 'Man City') & (df['result_class'] == 'L')].shape[0]\n```"} 

Executing Python:  
 ```python
man_united_wins = df[(df['home'] == 'Man United') & (df['result_class'] == 'W')].shape[0] + df[(df['away'] == 'Man United') & (df['result_class'] == 'L')].shape[0]
man_city_wins = df[(df['home'] == 'Man City') & (df['result_class'] == 'W')].shape[0] + df[(df['away'] == 'Man City') & (df['result_class'] == 'L')].shape[0]
``` 

Raw Result:  
 Man United won